In [1]:
from prepare_language_folder import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

In [2]:
data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'

If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [7]:
prepare_language_folder(data_collection_name)

No zipped data found for test_sessions. Proceeding with existing data.
No zipped data found for .DS_Store. Proceeding with existing data.
No zipped data found for 006_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 010_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 009_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 007_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for pilot_sessions. Proceeding with existing data.
No zipped data found for 008_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 011_SQ_CH_1_ET1. Proceeding with existing data.


In [4]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)
multipleye_sq

Folder test_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.


Parsing participant data : 011_SQ_CH_1_ET1: 100%|██████████| 6/6 [00:00<00:00, 724.49it/s]


data_collection_name	MultiplEYE_SQ_CH_Zurich_1_2025
num_sessions	6
num_pilots	0

In [5]:
multipleye_sq.create_sanity_check_report(overwrite=True)

Creating sanity check report for session 006_SQ_CH_1_ET1:   0%|          | 0/6 [00:00<?, ?it/s]

LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab
LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab


Creating sanity check report for session 010_SQ_CH_1_ET1:  17%|█▋        | 1/6 [00:28<02:24, 29.00s/it]

LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab
LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab


Creating sanity check report for session 009_SQ_CH_1_ET1:  33%|███▎      | 2/6 [01:00<02:00, 30.18s/it]

...

Creating sanity check report for session 008_SQ_CH_1_ET1:  67%|██████▋   | 4/6 [02:01<01:00, 30.35s/it]

LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab
LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab


Creating sanity check report for session 011_SQ_CH_1_ET1:  83%|████████▎ | 5/6 [02:28<00:29, 29.14s/it]

LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab
LWMC
PLAB
Flanker
Stroop
RAN
WikiVocab


Creating sanity check report for session 011_SQ_CH_1_ET1: 100%|██████████| 6/6 [02:55<00:00, 29.28s/it]
